In [1]:
import pandas as pd # para carregar o CSV
import numpy as np
from joblib import dump, load

from sklearn import tree
from sklearn.datasets import load_boston
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
sinais_vitais = pd.read_csv(r'sinais_vitais.txt')
sinais_vitais.set_index('id', inplace=True)

In [3]:
sinais_vitais.head()
sinais = sinais_vitais[['qPA','pulso', 'resp']].values
gravidade = sinais_vitais['grav'].values

In [4]:
sinais_p1 = sinais[:1300]
grav_p1 = gravidade[:1300]

In [5]:
predictions = []
#clf = clf.fit(sinais_p1, grav_p1)
#clf.predict([[4.745181,196.803149,11.875068],[1.707746,184.267283,14.256125]])

In [6]:
#k_fold com k=3 , utilizando a primeira e segunda parte
div = 3
sinais_k = np.array_split(sinais, div)
grav_k = np.array_split(gravidade, div)
clf = tree.DecisionTreeRegressor(max_depth=15)
clf = clf.fit(np.concatenate((sinais_k[0], sinais_k[1])), np.concatenate((grav_k[0], grav_k[1])))
prediction = clf.predict(sinais_k[2])

predictions.append(mean_squared_error(grav_k[2], prediction))

In [7]:
#k_fold com k=3 , utilizando a primeira e ultima parte
#testando com id=549 e id=550
clf = clf.fit(np.concatenate((sinais_k[0], sinais_k[2])), np.concatenate((grav_k[0], grav_k[2])))
prediction = clf.predict(sinais_k[1])

predictions.append(mean_squared_error(grav_k[1], prediction))

In [8]:
#k_fold com k=3 , utilizando a segunda e ultima parte
clf = clf.fit(np.concatenate((sinais_k[1], sinais_k[2])), np.concatenate((grav_k[1], grav_k[2])))
prediction = clf.predict(sinais_k[0])

predictions.append(mean_squared_error(grav_k[0], prediction))

In [9]:
predictions


[16.056594128150202, 18.69471902319624, 25.690398376553848]

In [10]:
# deve ter um nivel de aleatoriedade no algoritmo porque ele dá diferentes valores
print("MSE:", np.mean(predictions))

MSE: 20.14723717596676


In [11]:
dump(clf, 'cartRegre.joblib') 
clf2 = load('cartRegre.joblib')
pred =[]


pred.append(mean_squared_error(grav_k[0], clf2.predict(sinais_k[0])))
pred.append(mean_squared_error(grav_k[1], clf2.predict(sinais_k[1])))
pred.append(mean_squared_error(grav_k[2], clf2.predict(sinais_k[2])))

print("MSE:", np.mean(pred))

MSE: 8.580771741993834


In [12]:
# Carrega os Dadps
sinais_vitais = pd.read_csv(r'sinais_vitais_teste.txt')
sinais_vitais.set_index('id', inplace=True)
sinais = sinais_vitais[['qPA','pulso', 'resp']].values
grav = sinais_vitais['grav'].values

In [13]:
clf2 = load('cartRegre.joblib')

print("MSE:", mean_squared_error(grav, clf2.predict(sinais)))
print("MSE:", mean_absolute_error(grav, clf2.predict(sinais)))

MSE: 28.070270741350694
MSE: 3.0000988525252525


In [14]:
clf2.predict(sinais)
df = pd.DataFrame(clf2.predict(sinais), columns=['grav'])
df.to_csv('grav_cart.csv', index=False)